In this notebook we use multi-class classification models to predict the pitch outcome or result of a given pitcher, which are called events in pybaseball. We do a simple train-test split with validation.

In [86]:
# Importing libraries
import pybaseball
import pandas as pd
import numpy as np
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

# Enable cache because running pybaseball is a large query
pybaseball.cache.enable()

Here we set the pitch sequence length parameter. We will consider consecutive sequences of pitches of length pitch_sequence_length, and our objective is to predict the last pitch outcome using data about the previous pitch_sequence_length-1 pitches.

In [2]:
pitch_sequence_length=51

Upload the data.

In [3]:
train_start = '2023-04-01'
train_end = '2023-07-01'

validation_start = '2024-05-15'
validation_end = '2024-06-01'

test_start = '2025-05-15'
test_end = '2025-06-01'

# Train-test split with validation
train_data = pybaseball.statcast(start_dt=train_start, end_dt=train_end, verbose=True)
validation_data = pybaseball.statcast(start_dt=validation_start, end_dt=validation_end, verbose=True)
test_data = pybaseball.statcast(start_dt=test_start, end_dt=test_end, verbose=True)

This is a large query, it may take a moment to complete


100%|██████████| 92/92 [00:08<00:00, 10.66it/s]


This is a large query, it may take a moment to complete


100%|██████████| 18/18 [00:01<00:00, 10.19it/s]


This is a large query, it may take a moment to complete


100%|██████████| 18/18 [00:01<00:00, 10.88it/s]


In [4]:
# Remove any possible duplicate rows
train_data = train_data.drop_duplicates()
validation_data = validation_data.drop_duplicates()
test_data = test_data.drop_duplicates()

In [5]:
number_of_pitches = len(train_data) + len(validation_data) + len(test_data)
print(number_of_pitches)

499269


In [6]:
# New pitch data row is added to the top of the dataframe. Here we see this for a specific pitcher.
specific_pitcher_data = train_data[train_data['pitcher'].isin([605447])].copy()
print(specific_pitcher_data[['pitcher', 'game_date', 'at_bat_number', 'pitch_number']].head(40))

# Sort the pitching data in increasing time order by reversing the order of the rows
specific_pitcher_data = specific_pitcher_data[::-1]
print(specific_pitcher_data[['pitcher', 'game_date', 'at_bat_number', 'pitch_number']].head(40))


      pitcher  game_date  at_bat_number  pitch_number
1968   605447 2023-06-29             66             6
1979   605447 2023-06-29             66             5
2038   605447 2023-06-29             66             4
2097   605447 2023-06-29             66             3
2160   605447 2023-06-29             66             1
2249   605447 2023-06-29             65             1
2339   605447 2023-06-29             64             4
2379   605447 2023-06-29             64             3
2466   605447 2023-06-29             64             2
2551   605447 2023-06-29             64             1
2662   605447 2023-06-29             63             3
2735   605447 2023-06-29             63             2
2833   605447 2023-06-29             63             1
2952   605447 2023-06-29             62             6
3030   605447 2023-06-29             62             5
3133   605447 2023-06-29             62             4
3225   605447 2023-06-29             62             3
3337   605447 2023-06-29    

In [7]:
# Reorder the rows of pitching data by increasing time
train_data = train_data[::-1]
validation_data = validation_data[::-1]
test_data = test_data[::-1]

Features.

In [8]:
# Pitch count (or cumulative pitch number) of any pitch within a game
train_data.loc[:, 'pitch_count'] = train_data.groupby(['game_pk', 'pitcher']).cumcount() + 1
validation_data.loc[:, 'pitch_count'] = validation_data.groupby(['game_pk', 'pitcher']).cumcount() + 1
test_data.loc[:, 'pitch_count'] = test_data.groupby(['game_pk', 'pitcher']).cumcount() + 1

In [9]:
# Features

categorical_features = [
    'events',
    'pitcher',
    'batter',
    'pitch_type'
]

sort_features = ['game_date', 'at_bat_number', 'pitch_number']

count_features = [
    'pitch_count',
    'inning',
    'balls', 'strikes',
    'home_score', 'away_score', 'bat_score', 'fld_score',
    'post_home_score', 'post_away_score', 'post_bat_score'
]

continuous_features = [
    'release_speed',
    'release_pos_x',
    'release_pos_z',
    'vx0', 'vy0', 'vz0',
    'ax', 'ay', 'az',
    'effective_speed'
]

Data cleaning. Remove rows with nan event, since our goal is to predict the event. Remove columns which are deprecated or have no relation with pitch outcome.

In [10]:
def data_cleaner(data):
    # Only keep games occuring in the R = Regular Season, F = wild card, L = League Championship Series, W = World Series
    data = data[data['game_type'].isin(['R', 'F', 'L', 'W'])].copy()

    # Keep only relevant features
    relevant_data=data.loc[:, categorical_features + sort_features + count_features + continuous_features]

    # Remove rows with any nan or None values
    clean_data = relevant_data.dropna().copy()

    # Change count features to float32 to set them on the same footing as continuous features
    clean_data.loc[:, count_features] = clean_data[count_features].astype('float32')
    return(clean_data)

In [11]:
# Clean train, validation, and test
train_data = data_cleaner(train_data)
validation_data = data_cleaner(validation_data)
test_data = data_cleaner(test_data)

Turn the categorical variables into labels using a label encoder fitted on train.

In [12]:
train = train_data.copy()
validation = validation_data.copy()
test = test_data.copy()

In [13]:
label_encoders = {}
for cat_feature in categorical_features:
    all = pd.concat([train_data[cat_feature], validation_data[cat_feature], test_data[cat_feature]], axis=0)
    all = all.astype(str)
    
    label_encoder = LabelEncoder()
    label_encoder.fit(all)
    print(np.unique(label_encoder.transform(all)))
    label_encoders[cat_feature] = label_encoder
    
    train[cat_feature] = label_encoder.transform(train_data[cat_feature].astype(str))
    validation[cat_feature] = label_encoder.transform(validation_data[cat_feature].astype(str))
    test[cat_feature] = label_encoder.transform(test_data[cat_feature].astype(str))



[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 2

In [14]:
for cat_feature in categorical_features:
    print(label_encoders[cat_feature].classes_)

['catcher_interf' 'double' 'double_play' 'field_error' 'field_out'
 'fielders_choice' 'fielders_choice_out' 'force_out'
 'grounded_into_double_play' 'hit_by_pitch' 'home_run' 'sac_bunt'
 'sac_fly' 'sac_fly_double_play' 'single' 'strikeout'
 'strikeout_double_play' 'triple' 'truncated_pa' 'walk']
['425794' '425844' '434378' '445276' '445926' '446372' '448179' '450203'
 '453178' '453268' '453286' '455119' '456501' '456713' '458677' '458681'
 '471911' '472551' '472610' '476595' '477132' '488984' '489119' '489446'
 '493603' '500779' '502043' '502085' '502171' '502179' '502202' '502624'
 '506433' '506702' '517008' '518397' '518489' '518516' '518585' '518617'
 '518626' '518876' '518886' '519008' '519043' '519141' '519151' '519242'
 '519293' '519326' '521230' '523260' '527048' '527054' '534910' '541640'
 '542194' '542208' '542585' '542881' '542888' '542914' '542947' '543037'
 '543056' '543063' '543101' '543135' '543243' '543272' '543281' '543294'
 '543339' '543351' '543475' '543507' '543518' 

Standardize the continuous features using mean and standard deviation of training features.

In [15]:
# Compute mean and standard deviation of each feature, i.e. column) in train
train_standard_scaler = StandardScaler()
std_train=train.copy()

# Standardize train using mean and standard deviation of train features
std_train[continuous_features] = train_standard_scaler.fit_transform(train[continuous_features])

# Standardize validation using mean and standard deviation of train features
std_validation=validation.copy()
std_validation[continuous_features] = train_standard_scaler.fit_transform(validation[continuous_features])

# Standardize test using mean and standard deviation of train features
std_test=test.copy()
std_test[continuous_features] = train_standard_scaler.transform(test[continuous_features])

In [16]:
std_train

,events,pitcher,batter,pitch_type,game_date,at_bat_number,pitch_number,pitch_count,inning,balls,...,release_speed,release_pos_x,release_pos_z,vx0,vy0,vz0,ax,ay,az,effective_speed
3964,15,656,593,6,2023-04-01,1,4,4,1,1,...,1.221652,0.078617,0.835171,-0.157036,-1.212777,-2.332629,-0.416625,0.526775,1.998429,1.612225
3555,4,656,249,9,2023-04-01,2,4,8,1,0,...,-1.271244,-0.202918,0.744842,-0.265108,1.260620,1.061908,0.877800,-0.991526,-2.335675,-1.039743
3303,14,656,157,9,2023-04-01,3,2,10,1,0,...,-0.974077,-0.186357,0.726777,-0.331009,0.950876,-0.108199,0.865712,-1.039262,-1.665091,-0.718292
2612,19,656,139,14,2023-04-01,4,6,16,1,3,...,0.181570,0.034455,0.690645,0.326875,-0.180860,-1.460648,0.591241,-0.359955,-0.465180,0.583583
2176,15,656,270,9,2023-04-01,5,6,22,1,0,...,-0.990587,-0.247080,0.744842,-0.165814,0.964781,0.137508,0.683558,-1.505094,-1.959559,-0.637929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771,14,250,276,2,2023-07-01,74,3,33,9,1,...,-1.320772,-0.489973,0.202871,-0.026165,1.319252,0.829755,1.457955,-0.584314,-1.439129,-1.216540
4641,8,673,237,6,2023-07-01,75,2,2,9,0,...,1.155615,-0.539656,0.564185,0.705203,-1.161434,-0.724156,0.017435,0.416233,0.532295,1.162194
3766,19,196,466,13,2023-07-01,76,8,8,9,3,...,1.073069,0.051016,1.286814,-0.221373,-1.096408,-0.271525,-1.337985,1.092174,-0.781772,0.888961
3476,8,196,332,13,2023-07-01,77,3,11,9,0,...,1.337216,0.100699,1.395208,0.074045,-1.358589,-0.068274,-1.121739,1.591727,-0.610056,1.130049


Obtain the collection of all sliding window sequences of pitches per pitcher.

In [18]:
# Output is 3-dimensional numpy array of shape (number of windows, window size, input size=length of std_data rows)
def pitch_sequences(std_data, window_size):
# Group the data by pitchers
    groups = std_data.groupby('pitcher')

    # Create sliding windows of pitches per pitcher
    pitch_sequence_list=[]
    for _, group in groups:
        # Sort pitches (note that sort_values by default is ascending sorting order, that is, lexicographic)
        group = group.sort_values(by=sort_features)
        group = group.drop(columns=sort_features)
                
        # Create sliding windows
        for i in range(len(group) - window_size + 1):
            window = group.iloc[i:i + window_size]
            pitch_sequence_list.append(window.values)
            
    pitch_sequences = np.stack(pitch_sequence_list, axis=0)
    return(pitch_sequences)

In [19]:
train_pitch_sequences=pitch_sequences(std_train, pitch_sequence_length)
validation_pitch_sequences=pitch_sequences(std_validation, pitch_sequence_length)
test_pitch_sequences=pitch_sequences(std_test, pitch_sequence_length)

In [20]:
train_pitch_sequences

array([[[4, 0, 302, ..., 0.22771991888507132, -0.060788721139418474,
         -0.13968093139234145],
        [15, 0, 633, ..., -1.67794294264726, -2.1357649543973327,
         -2.663068028550858],
        [14, 0, 117, ..., -1.3945946038473502, -0.3726613580441981,
         -0.8629447490492409],
        ...,
        [1, 0, 18, ..., -2.3733020589492693, -0.39859094259745415,
         -0.8468722197679774],
        [4, 0, 577, ..., -0.9913512199741831, 0.34887475065480494,
         -0.18789851923613657],
        [4, 0, 611, ..., -2.0022361621497797, -1.975818899855186,
         -2.888083438488561]],

       [[15, 0, 633, ..., -1.67794294264726, -2.1357649543973327,
         -2.663068028550858],
        [14, 0, 117, ..., -1.3945946038473502, -0.3726613580441981,
         -0.8629447490492409],
        [4, 0, 385, ..., -0.36521044538988356, 0.27641286891973854,
         -0.011100697142226433],
        ...,
        [4, 0, 577, ..., -0.9913512199741831, 0.34887475065480494,
         -0.18789851

Extract labels and setup for model training and testing. All pairs (X,y) below have the following shape: X has shape (number of sequences, pitch_sequence_length-1, input_size=num_features) and y has shape (number of sequences,).

In [136]:
# Training, validation, and testing arrays
X_train = train_pitch_sequences[:, :, :-1].astype('float32')
y_train = train_pitch_sequences[:, -1, 0].astype(int)

X_validation = validation_pitch_sequences[:, :, :-1].astype('float32')
y_validation = validation_pitch_sequences[:, -1, 0].astype(int)

X_test = test_pitch_sequences[:, :, :-1].astype('float32')
y_test = test_pitch_sequences[:, -1, 0].astype(int)

# Training, validation, and testing tensors, which requires making everything float32
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_validation_tensor = torch.tensor(X_validation)
y_validation_tensor = torch.tensor(y_validation, dtype=torch.long)

X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [22]:
print(y_test.dtype)
for i in y_test:
    if np.array([i]).dtype != 'int64':
        print('bruh')
y_test_tensor.numpy().dtype

int64


dtype('int64')

For models like XGBoost and Random Forest which do not care about the time series aspect of the data, we flatten X into 2-dimensional array of shape (number of sequences, (pitch_sequence_length-1) x input_size=num_features).

In [23]:
flat_X_train=X_train.reshape(X_train.shape[0], -1)
flat_X_validation=X_validation.reshape(X_validation.shape[0], -1)
flat_X_test=X_test.reshape(X_test.shape[0], -1)

In [24]:
# The number of features decreased from 28 to 25 since in data_cleaner we removed sort_features
# The number of features in X_train decreases by 1 since we remove the last feature event to make y
print(std_train.shape, train_pitch_sequences.shape, X_train.shape, flat_X_train.shape)

(92404, 28) (62762, 51, 25) (62762, 51, 24) (62762, 1224)


We now implement various classification models below for the multiclass = events = pitch outcomes. For training we use cross entropy loss. For validation and testing we use AUC-ROC Macro-average loss because pitch outcomes are imbalanced with a heavy bias towards strikes and balls. 

In [25]:
num_classes = len(label_encoders['events'].classes_) # number of events = pitch outcomes
print(num_classes)


20


In [26]:
# F1 Macro evaluation metric
# y_pred_logits is numpy array of raw logits
# y_true is numpy array of same length with values the labels of label encoded events which are indices
def macro_f1_fn(y_pred_logits, y_true):
    y_pred_probabilities = F.softmax(torch.tensor(y_pred_logits), dim=1)
    y_pred = torch.argmax(y_pred_probabilities, dim=1).numpy()
    return f1_score(y_pred, y_true, average='macro')

Recurrent Neural Net.

In [27]:
# Create data sets
train_data_set = TensorDataset(X_train_tensor, y_train_tensor)
validation_data_set = TensorDataset(X_validation_tensor, y_validation_tensor)
test_data_set = TensorDataset(X_test_tensor, y_test_tensor)

In [28]:
input_size = X_train.shape[2] # number of features of X_train
print(input_size)

24


In [29]:
# Define the rnn model
class rnn_model(nn.Module):
    def __init__(self, hidden_size, num_layers):
        super(rnn_model, self).__init__()
        
        # RNN layers
        self.rnn = nn.RNN(input_size=input_size, 
                          hidden_size=hidden_size, 
                          num_layers=num_layers, 
                          batch_first=True)
        
        # Output layer
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Get RNN outputs (output, hidden state)
        rnn_output, _ = self.rnn(x)
        
        # Take the output from the last time step for classification
        last_output = rnn_output[:, -1, :]
        
        # Apply the fully connected layer
        output = self.fc(last_output)
        
        return output

In [37]:
# Define function with input the hyperparameters and output the Macro F1 score on validation
def rnn_validation_macro_f1(learning_rate, batch_size, hidden_size, num_layers):
    # Create DataLoader for Batch processing
    train_loader = DataLoader(train_data_set, batch_size=batch_size, shuffle=True)
        
    # Instantiate the model
    model = rnn_model(hidden_size=hidden_size, num_layers=num_layers)

    # Loss function for multi-class classification
    loss_fn = nn.CrossEntropyLoss()

    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    num_epochs = 20
    for epoch in range(num_epochs):
        model.train() 

        for X_input, y_true in train_loader:
            # Forward pass and calculate loss
            y_pred_logits = model(X_input)
            loss = loss_fn(y_pred_logits, y_true)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        # Print training progress
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.2f}")
        
    # Compute Macro F1 on validation set
    model.eval()
    with torch.no_grad():
        y_pred_logits = model(X_validation_tensor)
    return macro_f1_fn(y_pred_logits.numpy(), y_validation_tensor.numpy())

In [41]:
# Fine tune the hyperparameters by grid evaluation on validation
best_val_macro_f1 = 0
rnn_best_parameters = None
for learning_rate in [0.1]:
    for batch_size in [64, 128]:
        for hidden_size in [10]:
            for num_layers in [1]:
                val_macro_f1 = rnn_validation_macro_f1(learning_rate, batch_size, hidden_size, num_layers)
                if val_macro_f1 > best_val_macro_f1:
                    best_val_macro_f1 = val_macro_f1
                    rnn_best_parameters = (learning_rate, batch_size, hidden_size, num_layers)
print(rnn_best_parameters)

Epoch [10/20], Loss: 1.50
Epoch [20/20], Loss: 1.72
Epoch [10/20], Loss: 1.85
Epoch [20/20], Loss: 1.64
(0.1, 128, 10, 1)


In [43]:
# Evaluate the rnn model on testing
learning_rate, batch_size, hidden_size, num_layers = rnn_best_parameters

model = rnn_model(hidden_size=hidden_size, num_layers=num_layers)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

num_epochs=20
for epoch in range(num_epochs):
    model.train()
    # Forward pass and calculate loss
    y_pred_logits = model(X_train_tensor)
    loss = loss_fn(y_pred_logits, y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print training progress
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.2f}")


model.eval()  
with torch.no_grad():
    y_pred_logits = model(X_test_tensor)
macro_f1 = macro_f1_fn(y_pred_logits.numpy(), y_test_tensor.numpy())
print(f"Macro F1 score on test: {macro_f1:.4f}")

Epoch [10/20], Loss: 1.86
Epoch [20/20], Loss: 1.81
Macro F1 score on test: 0.0334


XGBoost. Warning, potential issues below.

In [ ]:
# Convert data to DMatrix, which is the optimal data structure for XGBoost
dtrain = xgb.DMatrix(flat_X_train, label=y_train)
dvalidation = xgb.DMatrix(flat_X_validation, label=y_validation)
dtest = xgb.DMatrix(flat_X_test, label=y_test)

In [105]:
# Define function with input the hyperparameters and output the Macro F1 score on validation
def xgb_validation_macro_f1(max_depth, learning_rate, n_estimators):
    model = XGBClassifier(
        objective = 'multi:softprob',
        num_class = num_classes,     
        max_depth = max_depth,
        eta = learning_rate,
        n_estimators = n_estimators,
        eval_metric = 'mlogloss'
    )
    model.fit(flat_X_train, y_train)
    y_pred_logits = model.predict_proba(flat_X_validation)
    return macro_f1_fn(y_pred_logits, y_validation)

In [106]:
best_macro_f1 = 0
xgb_best_parameters = {}
for max_depth in [3]:
    for learning_rate in [0.01]:
        for n_estimators in [10]:
            # Train the model and get the accuracy
            macro_f1 = xgb_validation_macro_f1(max_depth, learning_rate, n_estimators)
            #print(f"'Macro F1: {macro_f1:.2f}")
            
            # If the current model performs better, save the parameters
            if macro_f1 > best_macro_f1:
                best_macro_f1 = macro_f1
                xgb_best_parameters = (max_depth, learning_rate, n_estimators)
                print(macro_f1)
                        


1.0


In [128]:
model = XGBClassifier(
    objective='multi:softprob',  
    num_class=num_classes,             
    max_depth=4,              
    eta=0.1,                   
    eval_metric='mlogloss',     
    n_estimators=10,       
    random_state=42
)


In [130]:
model.fit(flat_X_train, y_train)

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'mlogloss'


In [137]:
y_pred_probabilities = model.predict_proba(flat_X_validation)
y_pred = torch.argmax(torch.tensor(y_pred_probabilities), dim=1).numpy()
print(y_pred)
y_test[0]=0
y_test[1]=0
print(f1_score(y_pred, y_validation, average='macro'))

[ 4 14  4 ... 15  1  4]
1.0


In [139]:
y_pred_probabilities = model.predict_proba(flat_X_test)
y_pred = torch.argmax(torch.tensor(y_pred_probabilities), dim=1).numpy()
print(y_pred)
print(f1_score(y_pred, y_test, average='macro'))

[15  4 15 ... 14  4  4]
0.9814158491812547


[15  4 15 ... 14  4  4]
0.9814158491812547


In [107]:
# Evaluate the rnn model on testing
model = xgb.train(
    xgb_best_parameters, 
    dtrain, 
    num_boost_round=100,
    evals=[(dtrain, 'train'), (dvalidation, 'valid')],
    early_stopping_rounds=10
)
y_pred_logits = model.predict(dvalidation, output_margin=True)
print(macro_f1_fn(y_pred_logits, y_validation))

AttributeError: 'tuple' object has no attribute 'copy'

Future directions.

Random forest.
Transformer.
Specific examples of pitchers and their pitch sequences. Maybe pick some from recent world series. Blue Jay's pitch mistake.